# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from config import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file=pd.read_csv('../weather_data.csv', index_col=0)
file.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind speed
0,Asopos,0,GR,1579829387,68,36.73,22.87,10.15,12.68
1,Bluff,77,NZ,1579829387,34,-46.60,168.33,25.61,1.89
2,Busselton,2,AU,1579829388,59,-33.65,115.33,22.22,0.89
3,Crib Point,18,AU,1579829388,59,-38.35,145.20,22.22,8.56
4,Rikitea,0,PF,1579829182,78,-23.12,-134.97,26.22,7.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = file[["Lat", "Lng"]]
# Fill NaN values and convert to float
humidity = file["Humidity"].astype(float)
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
file=file.loc[(file['Max Temp']>18)&(file['Wind speed']<10)&(file['Cloudiness']<10)]
file.dropna()
file.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind speed
2,Busselton,2,AU,1579829388,59,-33.65,115.33,22.22,0.89
4,Rikitea,0,PF,1579829182,78,-23.12,-134.97,26.22,7.50
19,Gualeguaychu,0,AR,1579829392,64,-33.01,-58.52,26.11,1.34
26,Cape Town,0,ZA,1579829333,68,-33.93,18.42,20.00,4.60
35,Carnarvon,0,AU,1579829190,21,-24.87,113.63,35.00,7.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel=file
hotel['Hotel name']=" "
hotel=hotel.reset_index()
hotel.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind speed,Hotel name
0,2,Busselton,2,AU,1579829388,59,-33.65,115.33,22.22,0.89,
1,4,Rikitea,0,PF,1579829182,78,-23.12,-134.97,26.22,7.50,
2,19,Gualeguaychu,0,AR,1579829392,64,-33.01,-58.52,26.11,1.34,
3,26,Cape Town,0,ZA,1579829333,68,-33.93,18.42,20.00,4.60,
4,35,Carnarvon,0,AU,1579829190,21,-24.87,113.63,35.00,7.20,


In [8]:
new_hotel=hotel.drop(['index'], axis=1)
new_hotel.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind speed,Hotel name
0,Busselton,2,AU,1579829388,59,-33.65,115.33,22.22,0.89,
1,Rikitea,0,PF,1579829182,78,-23.12,-134.97,26.22,7.50,
2,Gualeguaychu,0,AR,1579829392,64,-33.01,-58.52,26.11,1.34,
3,Cape Town,0,ZA,1579829333,68,-33.93,18.42,20.00,4.60,
4,Carnarvon,0,AU,1579829190,21,-24.87,113.63,35.00,7.20,


In [9]:
ten_row_dataframe=new_hotel.sample(10)
ten_row_dataframe

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind speed,Hotel name
12,Broken Hill,0,AU,1579829426,30,-31.95,141.43,21.00,3.60,
8,Santa Maria,0,BR,1579829410,78,-29.68,-53.81,23.00,5.10,
38,Rio Grande,0,BR,1579829484,79,-32.03,-52.10,21.98,6.99,
28,Bima,8,ID,1579829465,56,-8.47,118.72,30.63,2.61,
29,Matara,0,LK,1579829468,70,5.95,80.54,27.21,4.93,
48,Xai-Xai,3,MZ,1579829503,87,-25.05,33.64,20.93,0.32,
5,Lompoc,1,US,1579829399,82,34.64,-120.46,18.33,3.10,
33,Boyolangu,2,ID,1579829473,62,-8.12,111.89,30.99,1.21,
3,Cape Town,0,ZA,1579829333,68,-33.93,18.42,20.00,4.60,
35,Katobu,3,ID,1579829477,66,-4.94,122.53,30.98,1.55,


In [10]:
lat=[]
lng=[]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": f'{lat},{lng}', 
    "type": "lodging",
    "radius": 1500,
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in ten_row_dataframe.iterrows():

    lat=row['Lat']
    lng=row['Lng']
    params['location'] = f"{lat},{lng}"
    # assemble url and make API request
    response = requests.get(base_url, params=params).json() 
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        ten_row_dataframe.loc[index, 'Hotel name'] = results[0]['name']
        #new_hotel.loc[index, 'address'] = results[0]['vicinity']
        #new_hotel.loc[index, 'price_level'] = results[0]['price_level']
        #new_hotel.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Closest hotel is Broken Hill Tourist Park.
------------
Closest hotel is Hotel Morotin.
------------
Closest hotel is Swan Express Rio Grande.
------------
Closest hotel is Parewa Hotel.
------------
Closest hotel is Saffron Villa.
------------
Closest hotel is Platinum Hotel.
------------
Closest hotel is O'Cairns Inn & Suites.
------------
Closest hotel is Yos ega cv.
------------
Closest hotel is Southern Sun Cape Sun.
------------
Missing field/result... skipping.
------------


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ten_row_dataframe.iterrows()]
locations = ten_row_dataframe[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
hotel_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
# Display Map
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))